# 🚀 Analyse 1: Diachrone Frequenzdiagramme

## Hinweise zur Ausführung des Notebooks
Dieses **Notebook** kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../introduction/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht 
Im Folgenden werden die annotierten Dateien (CSV-Format) analysiert. Unser Ziel ist es, die Wort-/Lemma-Häufigkeiten des semantischen Felds "Luft" (eine vordefinierte Wortgruppe) über Zeit zu plotten und festzustellen, ob es, parallel zur industriellen Revolution, einen Anstieg im Auftreten des Felds gibt. 

Dafür werden folgendene Schritte durchgeführt:
1. Einlesen des Korpus, der Metadaten-Dateien für Korpora I und II und des semantischen Felds "Luft"
2. Extraktion der Worthäufigkeiten und Plotten der Worthäufigkeiten für Korpus I
3. Extraktion der Worthäufigkeiten und Plotten der Worthäufigkeiten für Korpus II
4. Diskussion der Ergebnisse

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks</b>
<ol>
<li> Installieren der Bibliotheken </li>
<li> Pfad zu den Daten setzen</li>
<li> Laden der Daten (z.B. über den Command `wget` (s.u.))</li>
</ol>
Zum Testen: Ausführen der Zelle "load libraries" und der Sektion "Einlesen der Daten". </br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [96]:
#  🚀 Install libraries 
! pip install pandas spacy tqdm plotly numpy

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [110]:
import re
import requests
from pathlib import Path
from typing import Dict, List, Union, Tuple

import pandas as pd
from time import time
from tqdm.auto import tqdm
from itables import show
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"

## Einlesen der Daten, Metadaten und der Grippe-Wortliste
Um eine/mehrere Dateien mit Python bearbeiten zu können, müssen die Dateien zuerst ausgewählt werden, d.h der [Pfad](https://en.wikipedia.org/wiki/Path_(computing)) zu den Dateien wird gesetzt, und dann eingelesen werden. 

### Einlesen des Korpus (CSV-Dateien)

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die CSV-Dateien gespeichert werden. Der Einfachheit halber wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/quadriga-dk/Text-Fallstudie-1/tree/main">GitHub Repository</a>, in dem die Daten gespeichert sind, vorausgesetzt. </br>
Danach werden alle CSV-Dateien im Korpus heruntergeladen und gespeichert. Dafür sind folgende Schritte nötig:
<ol>
    <li>Es wird eine Liste erstellt, die die URLs zu den einzelnen CSV-Dateien beinhaltet.</li>
    <li>Die Liste wird als txt-Datei gespeichert.</li>
    <li>Alle Dateien aus der Liste werden heruntergeladen und in dem Ordner <i>../data/csv</i> gespeichert.</li>
</ol>
Sollten die Dateien schon an einem anderen Ort vorhanden sein, können die Dateipfade zu den Ordnern angepasst werden. </br>
</details>

In [70]:
# 🚀 Create data directory path
corpus_dir = Path("../data/csv")
if not corpus_dir.exists():
    corpus_dir.mkdir()

In [5]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://api.github.com/repos/quadriga-dk/Text-Fallstudie-3/contents/data/spacy"
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_csv_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

TypeError: string indices must be integers, not 'str'

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all csv files – this step will take a while (ca. 7 minutes)
! wget -i github_csv_file_urls.txt -P ../data/spacy

Setzen des Pfads:

In [89]:
# set the path to csv files to be processed
corpus_dir = Path(r"../data/csv")

Einlesen der CSV-Dateien

In [92]:
annotated_docs = {}
start = time()
for fp in tqdm(corpus_dir.iterdir(), desc="Reading annotated data"):
    # check if the entry is a file, not a directory
    if fp.is_file():
        # check if the file has the correct suffix spacy
        if fp.suffix == '.csv':
            df = pd.read_csv(fp)
            annotated_docs[fp.stem] = df
took = time() - start
print(f"Loading the data took: {round(took, 4)} seconds") 

Reading annotated data: 0it [00:00, ?it/s]

Loading the data took: 6.1387 seconds


Wie viele Dateien wurden eingelesen?

In [93]:
len(annotated_docs)

656

Wie sieht der Anfang der ersten Datei aus?

In [107]:
annotated_docs[list(annotated_docs.keys())[1]][:15]

,Token,Lemma,POS
0,Fritz,Fritz,PROPN
1,Reuter,Reuter,PROPN
2,Ein,ein,DET
3,gräflicher,gräflich,ADJ
4,Geburtstag,Geburtstag,NOUN
5,.,--,PUNCT
6,Die,der,DET
7,Feier,Feier,NOUN
8,des,der,DET
9,Geburtstages,Geburtstag,NOUN


### Einlesen der Metadaten

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die Metadaten-Datei gespeichert wird. Wieder wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/quadriga-dk/Text-Fallstudie-1/tree/main">GitHub Repository</a> vorausgesetzt. </br>
Der Text wird aus GitHub heruntergeladen und in dem Ordner <i>../data/metadata/</i> abgespeichert. </br>
Der Pfad kann in der Variable <i>metadata_path</i> angepasst werden. Die einzulesende Datei muss die Endung `.csv` haben. </br>
</details>

In [20]:
# 🚀 Create metadata directory path
metadata_dir = Path("../metadata")
if not metadata_dir.exists():
    metadata_dir.mkdir()

In [ ]:
# 🚀 Load the metadata file from GitHub 
! wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-1/refs/heads/main/data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv -P ../data/metadata

In [102]:
# set path to metadata file
metadata_path_1 = '../metadata/metadata_corpus-german_language_fiction_1820-1900_50-per-decade.csv'
metadata_path_2 = '../metadata/metadata_corpus-german_language_fiction_1820-1900_50-per-decade_ALT.csv'

def read_replace_metadata(fp):
    corpus_metadata = pd.read_csv(fp)
    corpus_metadata['year'] = pd.to_datetime(corpus_metadata['year'], format='%Y')
    corpus_metadata = corpus_metadata.fillna("-")
    return corpus_metadata

# read metadata file to pandas dataframe
corpus_metadata_1 = read_replace_metadata(metadata_path_1)
corpus_metadata_2 = read_replace_metadata(metadata_path_2)

Wie sieht die Metadaten-Datei aus? (erste fünf Zeilen)

In [103]:
show(corpus_metadata_1)

Loading ITables v2.6.1 from the internet... (need help?)


### Einlesen der Wortliste (Semantisches Feld "Grippe")

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Parallel zur Metadaten-Datei wird ein Ordner für die Wortlisten-Datein angelegt, die Datei wird aus GitHub geladen und in dem erstellten Ordner abgelegt.
</details>

In [31]:
# 🚀 Create word list directory path
wordlist_dir = Path("../data/wordlist")
if not wordlist_dir.exists():
    wordlist_dir.mkdir()

In [ ]:
# 🚀 Load the wordlist file from GitHub 
! wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-1/refs/heads/main/data/wordlist/grippe.txt -P ../data/wordlist

In [39]:
path_to_wordlist = Path("../wordlist/semantic-field-luft.txt")
semantic_field_nouns = [word for word in path_to_wordlist.read_text().split("\n") if len(word) > 0]

Wie sieht die Wortliste aus?

In [40]:
semantic_field_nouns

['Luft',
 'Atem',
 'Wind',
 'Brise',
 'Hauch',
 'Atmosphäre',
 'Himmel',
 'Wolke',
 'Nebel',
 'Dunst',
 'Atemzug',
 'Odem',
 'Äther',
 'Duft',
 'Geruch',
 'Schwüle',
 'Frische']

## Suche nach einem Lemma und plotte die Häufigkeit
1. Relative Häufigkeit der Luft-Lemmata pro Text berechnen
2. Häufigkeiten plotten

### 1. Relative Häufigkeit der Luft-Lemmata pro Text berechnen

In [123]:
def extract_noun_list_frequencies(annotated_docs: Dict, metadata_df: pd.DataFrame, 
                                  noun_list: List[str]) -> pd.DataFrame:
    """
    Calculate the combined relative frequency of a list of nouns for each text.
    
    Parameters:
    -----------
    spacy_docs : dict
        Dictionary with file_ids as keys and spaCy Doc objects as values
    metadata_df : pd.DataFrame
        DataFrame with columns: 'lastname', 'firstname', 'title', 'year', 'ID', 'decade'
    noun_list : list of str
        List of noun lemmata to count together
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: filename, title, year, total_freq, total_count, total_tokens
    """
    results = []
    
    for meta_row in metadata_df.itertuples():
        file_id = meta_row.ID
        
        if file_id in annotated_docs:
            doc = annotated_docs[file_id]
        else:
            print(f"File {file_id} not in the corpus. Skipping...")
            continue
        
        # Count total tokens
        total_tokens = len(doc)

        # Skip empty texts
        if total_tokens == 0:
            continue
        
        # Count occurrences of any noun in the list
        total_count = sum(1 for row in doc.itertuples() 
                         if row.Lemma in noun_list
                         and row.POS == 'NOUN')
        
        # Calculate relative frequency per 100 tokens
        rel_freq = (total_count / total_tokens) * 100
        
        results.append({
            'filename': file_id,
            'lastname': meta_row.lastname,
            'firstname': meta_row.firstname,
            'title': meta_row.title,
            'year': meta_row.year,
            'relative_frequency': rel_freq,
            'total_count_semantic_field': total_count,
            'total_count_tokens': total_tokens
        })
    
    return pd.DataFrame(results)

In [124]:
# Extract frequencies for Corpus I
freq_1_df = extract_noun_list_frequencies(annotated_docs, corpus_metadata_1, semantic_field_nouns)

# Extract frequencies for Corpus II
freq_2_df = extract_noun_list_frequencies(annotated_docs, corpus_metadata_2, semantic_field_nouns)

#### Ergebnisse für Korpus I angucken

In [117]:
show(freq_1_df)

Loading ITables v2.6.1 from the internet... (need help?)


#### Ergebnisse für Korpus II angucken

In [118]:
show(freq_2_df)

Loading ITables v2.6.1 from the internet... (need help?)


### Häufigkeiten plotten

In [131]:
def plot_noun_list_scatter(freq_df: pd.DataFrame, noun_list: List[str], 
                          show_trendline: bool = True, verbose: bool = False):
    """
    Create a scatter plot showing the combined frequency of a noun list over time.
    
    Parameters:
    -----------
    freq_df : pd.DataFrame
        DataFrame returned by extract_noun_list_frequencies()
    noun_list : list of str
        The list of nouns being analyzed (for the text)
    show_trendline : bool
        If True, add a linear regression trendline (default: True)
    verbose : bool
        If True, print diagnostic information about the trendline (default: False)
    
    Returns:
    --------
    plotly.graph_objects.Figure
        The figure object (will display automatically in Jupyter)
    """
    # Create scatter plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=freq_df['year'],
        y=freq_df['relative_frequency'],
        mode='markers',
        name='Texts',
        text=freq_df['title'],
        customdata=np.column_stack((freq_df['total_count_semantic_field'], freq_df['lastname'])),
        hovertemplate='<b>%{text}</b> (%{customdata[1]})<br>' +
                     'Year: %{x|%Y}<br>' +
                     'Frequency: %{y:.2f} per 100 tokens<br>' +
                     'Total count: %{customdata[0]}<br>' +
                     '<extra></extra>',
        marker=dict(
            size=8,
            color='steelblue',
            opacity=0.7,
            line=dict(width=1, color='white')
        )
    ))
    
    # Add trendline if requested
    if show_trendline:
        # Calculate linear regression
        
        
        # Convert year to numeric, handling datetime objects
        if pd.api.types.is_datetime64_any_dtype(freq_df['year']):
            # If datetime, extract the year
            x = freq_df['year'].dt.year.values.astype(float)
        else:
            # Otherwise convert to numeric
            x = pd.to_numeric(freq_df['year'], errors='coerce').values
        
        y = freq_df['relative_frequency'].values
        
        # Remove any NaN values
        valid_idx = ~(np.isnan(x) | np.isnan(y))
        x = x[valid_idx]
        y = y[valid_idx]
        
        if verbose:
            print(f"Valid data points for trendline: {len(x)}")
            print(f"Year range: {x.min():.0f} to {x.max():.0f}")
            print(f"Frequency range: {y.min():.2f} to {y.max():.2f}")
        
        if len(x) > 1:  # Need at least 2 points for a line
            # Fit line: y = mx + b
            m, b = np.polyfit(x, y, 1)
            
            if verbose:
                print(f"Trendline equation: y = {m:.6f}x + {b:.4f}")
                print(f"Slope interpretation: {'increasing' if m > 0 else 'decreasing' if m < 0 else 'flat'} trend")
            
            # Create trendline
            x_trend = np.array([x.min(), x.max()])
            y_trend = m * x_trend + b
            
            fig.add_trace(go.Scatter(
                x=x_trend,
                y=y_trend,
                mode='lines',
                name='Trend',
                line=dict(color='red', width=2, dash='dash'),
                hovertemplate='Trendline<br>Year: %{x}<br>%{y:.2f}<extra></extra>'
            ))
        else:
            if verbose:
                print("Warning: Not enough valid data points to draw trendline (need at least 2)")
    
    # Create a readable noun list for the title
    noun_list = list(noun_list)
    noun_list_str = ', '.join(noun_list[:5])
    if len(noun_list) > 5:
        noun_list_str += f', ... ({len(noun_list)} total)'
    
    # Update layout
    fig.update_layout(
        title=f'Combined Frequency of Noun List [{noun_list_str}]',
        xaxis_title='Year',
        yaxis_title='Relative Frequency (per 100 tokens)',
        hovermode='closest',
        height=600,
        showlegend=show_trendline
    )
    
    return fig

In [132]:
# Create scatter plot
plot_noun_list_scatter(freq_1_df, semantic_field_nouns)

In [133]:
# Create scatter plot
plot_noun_list_scatter(freq_2_df, semantic_field_nouns)

## Schreiben der Ergebnisse 
Die Ergebnisse werden als Tabellen (`.csv`-Dateien) gespeichert, so kann die Erstellung der Abbildungen unbahängig von diesem Notebook nachvollzogen werden und die werden Ergebnisse so nachnutzbar.

In [122]:
result_dir = Path("results/")
if not result_dir.exists():
    result_dir.mkdir()

result_path_1 = result_dir / "results-corpus-german_language_fiction_1820-1900_50-per-decade-I.csv"
result_path_2 = result_dir / "results-corpus-german_language_fiction_1820-1900_50-per-decade-II.csv"

freq_1_df.to_csv(result_path_1, index=False)
freq_2_df.to_csv(result_path_2, index=False)

## Diskussion der Häufigkeitsanalyse

Die Analyse der relativen Häufigkeiten zeigt, dass deutschsprachige literarische Texte des 19. Jahrhunderts **nicht** vermehrt über Luft und Luft-verwandte Begriffe sprechen, sondern der Trend in beiden Subkorpora stagniert. Zwar gibt es Ausreißer wie z.B. *Das Dorf im Gebirge* (von Hofmannsthal) oder *Das Schattenspiel* von Flaischlen, die beide 

Es gibt mehrere Deutungsansätze dieses Ergebnisses: Zum einen kann es sein, dass deutschsprachige Literatur die durch die revolutionelle Revolution herbeigeführte Veränderung der Luftqualität tatsächlich nicht reflektiert. Zum anderen ist es möglich, dass unsere Operationalisierung zu kurz gegriffen ist, z.B. da abnehmende Luftqualität subtiler angedeutet werden könnte oder die relative Häufigkeit keine gute Metrik für die Importanz der Thematik ist. 